In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time as sleep
import pyautogui as pos
from openpyxl import load_workbook
import win32com.client as win32
from datetime import datetime

def print_start_time():
    start_time = datetime.now()
    print('Automation start time:', start_time.strftime("Date: %m/%d/%y - Time: %H:%M:%S"))
    return start_time

def login_and_download_report():
    browser = webdriver.Chrome()
    browser.maximize_window()
    browser.get("https://empresa.rastreamento.com.br/rastreador/login.xhtml")
    browser.find_element(By.XPATH, "//*[contains(text(),'Fechar')]").click()
    browser.find_element(By.XPATH, '//*[@id="j_username"]').send_keys("000cnpj000")
    browser.find_element(By.XPATH, '//*[@id="password"]').send_keys("3#4*4213*")
    browser.find_element(By.XPATH, "//*[contains(text(),'Entrar')]").click()
    browser.find_element(By.XPATH, '//*[@id="menuform:consult"]/a').click()
    browser.find_element(By.XPATH, '//*[@id="menuform:position"]/a').click()
    browser.find_element(By.XPATH, '//*[@id="selectionForm:selectionType_label"]').click()
    browser.find_element(By.XPATH, '//*[@id="selectionForm:selectionType_0"]').click()

    coordinates = [
        (1270, 356), (1216, 560), (107, 707), (1188, 88),
        (27, 65), (73, 332), (1195, 175), (665, 197), 
        (1262, 174), (1342, 28)
    ]

    for x, y in coordinates:
        pos.sleep(2)
        pos.moveTo(x, y)
        pos.click(x, y)

    browser.close()

def process_excel_file(filepath):
    workbook = load_workbook(filename=filepath)
    sheet = workbook['positions-report']

    for i in range(4, len(sheet['D']) + 1):
        address = sheet[f'D{i}'].value
        sheet[f'D{i}'].value = address.split(',')[-1]
    
    workbook.save(filename=filepath)
    new_filepath = filepath.replace("ultimas-posicoes.xlsx", "Status_Frota.xlsx")
    workbook.save(new_filepath)
    return new_filepath

def send_email(attachment_path):
    outlook = win32.Dispatch('outlook.application')
    email = outlook.CreateItem(0)
    email.To = "monitoramento@empresa123.com.br"
    email.Subject = "Fleet Status Update"
    email.HTMLBody = """
    <p>Hello,</p>
    <p>Attached is the fleet status report.</p>
    <p><img src="D:\\Users\\Geral\\Documents\\EMAIL_SIGNATURE.png">.</p>
    """
    email.Attachments.Add(attachment_path)
    email.Send()

def print_end_time(start_time):
    end_time = datetime.now()
    print('Automation end time:', end_time.strftime("Date: %m/%d/%y - Time: %H:%M:%S"))
    duration = end_time - start_time
    print('Email sent successfully - Execution time:', duration)

def main():
    start_time = print_start_time()
    login_and_download_report()
    filepath = "D:\\Users\\Geral\\Downloads\\ultimas-posicoes.xlsx"
    new_filepath = process_excel_file(filepath)
    send_email(new_filepath)
    print_end_time(start_time)

if __name__ == "__main__":
    main()
    